<ol>
<li>Load the dataset as a Spark DataFrame.</li>

<li>Print schema and count nulls per column.</li>

<li>Identify the number of distinct products and customers.</li>

<li>How many records have missing CustomerID or Description?</li>

<li>Filter only valid records (Quantity > 0 and UnitPrice > 0).</li>

<li>Save the cleaned DataFrame as a Bronze Parquet file.</li>
</ol>

In [0]:
df=spark.read.format("csv")\
    .option("header","true")\
    .option("inferSchema","true")\
    .load("/Volumes/workspace/default/abc/OnlineRetail.csv")
display(df)

In [0]:
df.printSchema()

In [0]:
from pyspark.sql import functions as F
null_counts=df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c)
                        for c in df.columns])
null_counts.show()

In [0]:
df.select('StockCode').distinct().count()
# df.select('CustomerID').distinct().count()

In [0]:
df.select('CustomerID').distinct().count()

In [0]:
missing_records=df.filter(F.col('CustomerID').isNull()|F.col('Description').isNull())
missing_records_count = missing_records.count()
print(f"Number of records with missing CustomerID or Description: {missing_records_count}")

In [0]:
df_filtered=df.filter(
    (F.col('Quantity')>0) & (
        F.col('UnitPrice')>0)
)



In [0]:
dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get()

In [0]:
bronze_path = "/Volumes/workspace/default/abc/retail_filtered.parquet"

df_filtered.write.mode("overwrite").parquet(bronze_path)